In [1]:
import os
from dotenv import find_dotenv, load_dotenv
from pathlib import Path
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

absProjectDir = Path(os.getcwd()).resolve().parents[0]
projectDir = os.path.relpath(absProjectDir,os.curdir)
load_dotenv(find_dotenv())

trainDfPath = os.path.join(projectDir, os.environ.get("REF_PROC_TRAIN_DF"))
testDfPath = os.path.join(projectDir, os.environ.get("REF_PROC_TEST_DF"))
testOrigDir = os.path.join(projectDir, os.environ.get("PROC_TEST_ORIG_DIR"))
testAugmDir = os.path.join(projectDir, os.environ.get("PROC_TEST_AUG_DIR"))
trainOrigDir = os.path.join(projectDir, os.environ.get("PROC_TRAIN_ORIG_DIR"))
trainAugmDir = os.path.join(projectDir, os.environ.get("PROC_TRAIN_AUG_DIR"))
testRootDir = os.path.commonpath([testOrigDir, testAugmDir])
trainRootDir = os.path.commonpath([trainOrigDir, trainAugmDir])

sys.path.append(os.path.join(projectDir,'src/data'))
import imggen as imgen

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

class ConvBatchNorm(layers.Layer):
    def __init__(self,filters, kernel_size = (3,3), strides=(1, 1)) :
        super(ConvBatchNorm,self).__init__()
        self.Layer1 = Conv2D(filters,kernel_size,strides,padding = "same")
        self.Layer2 = BatchNormalization()
    def call(self,inputs,training = None):
        x = self.Layer1(inputs)
        x = self.Layer2(x,training = training)
        return tf.nn.relu(x)
    
class InceptionV1(layers.Layer):
    def __init__(self, filters1, filters31, filters33, filters51, filters55, filtersPool):
        super(InceptionV1, self).__init__()
        self.Layer11 = ConvBatchNorm(filters1, (1,1), (1,1))
        self.Layer12 = ConvBatchNorm(filters31, (1,1), (1,1))
        self.Layer13 = ConvBatchNorm(filters51, (1,1), (1,1))
        self.Layer14 = MaxPool2D(pool_size=(3, 3), strides=(1,1), padding = "same")
        self.Layer22 = ConvBatchNorm(filters33, (3,3), (1,1))
        self.Layer23 = ConvBatchNorm(filters55,(5,5),(1,1))
        self.Layer24 = ConvBatchNorm(filtersPool, (1,1), (1,1))
        self.Layer3 = Concatenate()
        self.Layer4 = ReLU()
        
    def call(self, inputs, training = None):
        x1 = self.Layer11(inputs,training)
        x2 = self.Layer12(inputs,training)
        x2 = self.Layer22(x2,training)
        x3 = self.Layer13(inputs,training)
        x3 = self.Layer23(x3, training)
        x4 = self.Layer14(inputs)
        x4 = self.Layer24(x4, training)
        x5 = self.Layer3([x1,x2,x3,x4])
        x5 = self.Layer4(x5)
        return x5

class CustomInceptionV1(layers.Layer):
    def __init__(self, filters, r0):
        super(CustomInceptionV1,self).__init__()
        f1 = int(r0 * filters  * 0.25)
        f31 = int(r0 * filters)
        f33 = f31
        f51 = int(r0 * filters * 0.15)
        f55 = f51
        fPool = filters - (f1 + f33 + f55)
        self.InceptionLayer = InceptionV1(f1, f31, f33, f51, f55, fPool)
    
    def call(self,inputs,training = None):
        return self.InceptionLayer.call(inputs,training)
    
def CustomInceptionModel1(r = 0.5, kernel = (3,3), filters = 32, units = 512, input_shape = (224,224,3)):
    model = Sequential(name = "CustomInceptionV1_R{}_Kern{}_Filters{}_Units{}".format(int(r*10),kernel[0],filters,units))
    model.add(InputLayer(input_shape))
    layers = []
    layers.append(ConvBatchNorm(filters, kernel, (1,1)))
    layers.append(ConvBatchNorm(filters * 2, kernel, (1,1)))
    layers.append(CustomInceptionV1(filters * 4,r))
    layers.append(CustomInceptionV1(filters * 8, r))
    layers.append(CustomInceptionV1(filters * 16, r))
    for layer in layers:
        model.add(layer)
        model.add(MaxPool2D(pool_size=(2, 2), strides = (2,2)))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(units, activation = 'relu'))
    model.add(Dense(1,activation = 'sigmoid'))
    model.compile(optimizer=Adam(lr=0.0001),loss=BinaryCrossentropy(),metrics=['accuracy'])
    return model

In [77]:
trgen = imgen.Generator(trainDfPath, trainRootDir, (224,224))
trgen.SetDataFrameForGeneration([[2],[3]],[[2048],[2048]])

tsgen = imgen.Generator(testDfPath,testRootDir, (224,224))
tsgen.SetDataFrameForGeneration([[2],[3]], [[1024],[1024]])

In [78]:
train_set = trgen.GetGenerator(bs = 16)
test_set = tsgen.GetGenerator(bs = 16)

Found 4096 validated image filenames belonging to 2 classes.
Found 2048 validated image filenames belonging to 2 classes.


In [79]:
model = CustomInceptionModel1()

In [80]:
hist = model.fit(train_set,
                 steps_per_epoch = 2048 / 8,
                 epochs = 20, validation_data = test_set,validation_steps = 1024 / 8)

Train for 256.0 steps, validate for 128.0 steps
Epoch 1/20
256/256 [==============================] - 449s 2s/step - loss: 0.6163 - accuracy: 0.6736 - val_loss: 1.7138 - val_accuracy: 0.5000
Epoch 2/20
256/256 [==============================] - 410s 2s/step - loss: 0.5817 - accuracy: 0.6963 - val_loss: 0.6533 - val_accuracy: 0.6313
Epoch 3/20
256/256 [==============================] - 410s 2s/step - loss: 0.5653 - accuracy: 0.7129 - val_loss: 0.6635 - val_accuracy: 0.6118
Epoch 4/20
256/256 [==============================] - 409s 2s/step - loss: 0.5455 - accuracy: 0.7246 - val_loss: 1.0366 - val_accuracy: 0.5288
Epoch 5/20
256/256 [==============================] - 458s 2s/step - loss: 0.5223 - accuracy: 0.7424 - val_loss: 1.1638 - val_accuracy: 0.5093
Epoch 6/20
256/256 [==============================] - 412s 2s/step - loss: 0.4940 - accuracy: 0.7593 - val_loss: 0.6578 - val_accuracy: 0.6382
Epoch 7/20
256/256 [==============================] - 566s 2s/step - loss: 0.4586 - accuracy: 

In [73]:
256 * 3 /8

96.0

In [30]:
class FooModel(Model):
    def __init__(self,name = "FooModel", **kwargs):
        super(FooModel, self).__init__(name = name, **kwargs)
        self.layer1 = InputLayer((224,224,3))
        self.layer2 = ConvBatchNorm(32, (3,3), (1,1))
        self.layer3 = CustomInceptionV1(128, 0.5)
        self.layer33 = MaxPool2D(pool_size=(2, 2), strides = (2,2))
        self.layer4 = GlobalAveragePooling2D()
        self.layer5 = Dense(256, activation = 'relu')
        self.layer6 = Dense(1,activation = 'sigmoid')
        
    def call(self, inputs, training = None):
        x = self.layer1(inputs)
        x = self.layer2(x,training)
        x = self.layer3(x,training)
        x = self.layer33(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        return x

In [21]:
class FooModel(Sequential):
    def __init__(self,name = "FooModel", **kwargs):
        super(FooModel, self).__init__(name = name, **kwargs)
        self.layers = []
        self.layers.append(InputLayer((224,224,3)))
        self.layers.append(ConvBatchNorm(32, (3,3), (1,1)))
        self.layers.append(CustomInceptionV1(128, 0.5))
        self.layers.append(GlobalAveragePooling2D())
        self.layers.append(Dense(256, activation = 'relu'))
        self.layers.append(Dense(1,activation = 'sigmoid'))
        
    def call(self, inputs, training = None):
        x = self.layers[0](inputs)
        for i in range(1,7):
            x = self.layers[i](x)
        return x

In [29]:
model = Sequential()
model.add(InputLayer((224,224,3)))
model.add(ConvBatchNorm(32, (3,3), (1,1)))
model.add(CustomInceptionV1(128, 0.5))
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))

In [30]:
#model = FooModel()
model.compile(optimizer=Adam(lr=0.0001),loss=BinaryCrossentropy(),metrics=['accuracy'])

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_batch_norm_42 (ConvBatc (None, 224, 224, 32)      1024      
_________________________________________________________________
custom_inception_v1_6 (Custo (None, 224, 224, 128)     43990     
_________________________________________________________________
global_average_pooling2d_6 ( (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 257       
Total params: 78,295
Trainable params: 77,829
Non-trainable params: 466
_________________________________________________________________


In [ ]:
hist = model.fit(train_set,
                 steps_per_epoch = 128 / 4,
                 epochs = 20, validation_data = test_set,validation_steps = 1)

In [61]:
r = 0.5
filters = 32
kernel = (3,3)
units = 512
model = Sequential()#name = "CustomInceptionV1_R{}_Kern{}_Filters{}_Units{}".format(int(r*10),kernel[0],filters,units))
model.add(InputLayer((224,224,3)))
model.add(ConvBatchNorm(filters, kernel, (1,1)))
model.add(MaxPool2D(pool_size=(2, 2), strides = (2,2)))
model.add(ConvBatchNorm(filters * 2, kernel, (1,1)))
model.add(MaxPool2D(pool_size=(2, 2), strides = (2,2)))
model.add(CustomInceptionV1(filters * 4,r))
model.add(MaxPool2D(pool_size=(2, 2), strides = (2,2), padding='same'))
model.add(CustomInceptionV1(filters * 8, r))
model.add(MaxPool2D(pool_size=(2, 2), strides = (2,2)))
model.add(CustomInceptionV1(filters * 16, r))
model.add(MaxPool2D(pool_size=(2, 2), strides = (2,2)))
model.add(GlobalAveragePooling2D())
model.add(Dense(units, activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(optimizer=Adam(lr=0.0001),loss=BinaryCrossentropy(),metrics=['accuracy'])

In [71]:
hist = model.fit(train_set,
                 steps_per_epoch = 128 / 4,
                 epochs = 1, validation_data = test_set,validation_steps = 1)

Train for 32.0 steps, validate for 1 steps
32/32 [==============================] - 56s 2s/step - loss: 0.6080 - accuracy: 0.6836 - val_loss: 0.7218 - val_accuracy: 0.2500


In [64]:
model.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_batch_norm_315 (ConvBat (None, 224, 224, 32)      1024      
_________________________________________________________________
max_pooling2d_139 (MaxPoolin (None, 112, 112, 32)      0         
_________________________________________________________________
conv_batch_norm_316 (ConvBat (None, 112, 112, 64)      18752     
_________________________________________________________________
max_pooling2d_140 (MaxPoolin (None, 56, 56, 64)        0         
_________________________________________________________________
custom_inception_v1_44 (Cust (None, 56, 56, 128)       48086     
_________________________________________________________________
max_pooling2d_141 (MaxPoolin (None, 28, 28, 128)       0         
_________________________________________________________________
custom_inception_v1_45 (Cust (None, 28, 28, 256)     